In [1]:
import re
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from langchain.text_splitter import TokenTextSplitter
from langchain.document_loaders import DiffbotLoader
from secret import *

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
from loader.radiopaedia import RadioWebLoader
search_query = "Atelectasis"
radio_raw_documents = RadioWebLoader(search_query).load()
wiki_raw_documents = WikipediaLoader(query="Atelectasis").load()
raw_documents = radio_raw_documents + wiki_raw_documents

c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [3]:
len(radio_raw_documents), len(wiki_raw_documents)

(20, 24)

In [4]:
text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)
documents = text_splitter.split_documents(
    raw_documents
)  # should I only take the first? which is the most relevant?

In [5]:
from langchain.indexes import GraphIndexCreator
from langchain.graphs.networkx_graph import KnowledgeTriple
from langchain.llms import OpenAI

In [6]:
index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))

c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAI instead.
  warn_deprecated(


In [7]:
graph = None
for d in documents:
    g_temp = index_creator.from_text(d.page_content)
    if graph is None:
        graph = g_temp
    else:
        triplets = graph.get_triples()
        for t in triplets:
            graph.add_triple(knowledge_triple=KnowledgeTriple(*t))

In [8]:
from langchain.chains import GraphQAChain

In [9]:
chain = GraphQAChain.from_llm(OpenAI(temperature=0), graph=graph, verbose=True)

In [10]:
lesion = "Atelectasis"
questions = [
    f"What is {lesion}?",
    f"What are the symptoms associated with {lesion}?",
    f"What can cause {lesion}?",
    f"What are the patient’s symptoms that are relevant for {lesion}?",
    f"What are the relevant clinical signs for the etiological diagnosis of {lesion}?",
    f"What are the relevant laboratory data for the etiological diagnosis of {lesion}?",
    f"What are the relevant clinical characteristics for the etiological diagnosis of {lesion}",
    f"What are the patient’s personal relevant history for the etiological diagnosis of {lesion}",
]

In [11]:
res_dict = {}
for q in questions:
    res = chain.run(q)
    res_dict[q] = res.strip()

c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis describes small areas of collapsed lung
Atelectasis is usually seen on chest x-rays
Atelectasis small areas of collapsed lung describes
Atelectasis chest x-rays is usually seen on

> Finished chain.


> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis describes small areas of collapsed lung
Atelectasis is usually seen on chest x-rays
Atelectasis small areas of collapsed lung describes
Atelectasis chest x-rays is usually seen on

> Finished chain.


> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis describes small areas of collapsed lung
Atelectasis is usually seen on chest x-rays
Atelectasis small areas of collapsed lung describes
Atelectasis chest x-rays is usually seen on

> Finished chain.


> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis describes s

In [12]:
for q in questions:
    print(res_dict[q])

Atelectasis is a condition where small areas of the lung collapse, and it is typically seen on chest x-rays.
The knowledge triplets do not provide information about symptoms associated with Atelectasis.
There are several potential causes of atelectasis, including blockage of the airway, lung diseases, and surgery.
The patient may experience difficulty breathing, chest pain, and a cough.
The relevant clinical signs for the etiological diagnosis of Atelectasis are small areas of collapsed lung and the presence of atelectasis on chest x-rays.
There is no specific laboratory data for the etiological diagnosis of Atelectasis. It is usually diagnosed through imaging techniques such as chest x-rays or CT scans. Other diagnostic tests, such as blood tests or pulmonary function tests, may be used to determine the underlying cause of the atelectasis.
The relevant clinical characteristics for the etiological diagnosis of Atelectasis are small areas of collapsed lung and being usually seen on ches

In [13]:
raise StopIteration()

StopIteration: 

In [ ]:
chain.run("What is Atelectasis?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis describes small areas of collapsed lung
Atelectasis is caused by adjacent compression
Atelectasis is caused by passive atelectasis
Atelectasis is caused by dependent atelectasis
Atelectasis is usually seen on chest x-rays as small volume linear shadows
Atelectasis is often helpful to determine the cause or confirm that there is no proximal obstruction CT is
Atelectasis small areas of collapsed lung describes
Atelectasis dependent atelectasis is caused by
Atelectasis small volume linear shadows is usually seen on chest x-rays as
Atelectasis CT is is often helpful to determine the cause or confirm that there is no proximal obstruction

> Finished chain.


' Atelectasis is a condition in which small areas of the lung collapse due to adjacent compression, passive atelectasis, or dependent atelectasis. It is usually seen on chest x-rays as small volume linear shadows and CT is often helpful to determine the cause or confirm that there is no proximal obstruction.'

In [ ]:
chain.run("What are the symptoms of Atelectasis?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis describes small areas of collapsed lung
Atelectasis is caused by adjacent compression
Atelectasis is caused by passive atelectasis
Atelectasis is caused by dependent atelectasis
Atelectasis is usually seen on chest x-rays as small volume linear shadows
Atelectasis is often helpful to determine the cause or confirm that there is no proximal obstruction CT is
Atelectasis small areas of collapsed lung describes
Atelectasis dependent atelectasis is caused by
Atelectasis small volume linear shadows is usually seen on chest x-rays as
Atelectasis CT is is often helpful to determine the cause or confirm that there is no proximal obstruction

> Finished chain.


' Atelectasis is usually asymptomatic, but some people may experience shortness of breath, chest pain, and coughing.'

In [ ]:
chain.run("What can cause Atelectasis?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis describes small areas of collapsed lung
Atelectasis is caused by adjacent compression
Atelectasis is caused by passive atelectasis
Atelectasis is caused by dependent atelectasis
Atelectasis is usually seen on chest x-rays as small volume linear shadows
Atelectasis is often helpful to determine the cause or confirm that there is no proximal obstruction CT is
Atelectasis small areas of collapsed lung describes
Atelectasis dependent atelectasis is caused by
Atelectasis small volume linear shadows is usually seen on chest x-rays as
Atelectasis CT is is often helpful to determine the cause or confirm that there is no proximal obstruction

> Finished chain.


' Atelectasis can be caused by adjacent compression, passive atelectasis, and dependent atelectasis.'

In [ ]:
chain.run("What clinical features radiologists need for diagnosing Atelectasis?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis describes small areas of collapsed lung
Atelectasis is caused by adjacent compression
Atelectasis is caused by passive atelectasis
Atelectasis is caused by dependent atelectasis
Atelectasis is usually seen on chest x-rays as small volume linear shadows
Atelectasis is often helpful to determine the cause or confirm that there is no proximal obstruction CT is
Atelectasis small areas of collapsed lung describes
Atelectasis dependent atelectasis is caused by
Atelectasis small volume linear shadows is usually seen on chest x-rays as
Atelectasis CT is is often helpful to determine the cause or confirm that there is no proximal obstruction

> Finished chain.


' Radiologists need to look for small areas of collapsed lung, small volume linear shadows, and adjacent compression on chest x-rays to diagnose Atelectasis. CT scans may also be helpful to determine the cause or confirm that there is no proximal obstruction.'

In [ ]:
chain.run("what are radiographic features of Atelectasis?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Atelectasis
Full Context:
Atelectasis describes small areas of collapsed lung
Atelectasis is caused by adjacent compression
Atelectasis is caused by passive atelectasis
Atelectasis is caused by dependent atelectasis
Atelectasis is usually seen on chest x-rays as small volume linear shadows
Atelectasis is often helpful to determine the cause or confirm that there is no proximal obstruction CT is
Atelectasis small areas of collapsed lung describes
Atelectasis dependent atelectasis is caused by
Atelectasis small volume linear shadows is usually seen on chest x-rays as
Atelectasis CT is is often helpful to determine the cause or confirm that there is no proximal obstruction

> Finished chain.


' Atelectasis is usually seen on chest x-rays as small volume linear shadows.'